### Login to Huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Load the model

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", trust_remote_code=True)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, 
    llm_int8_enable_fp32_cpu_offload=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [21]:
next(model.parameters()).dtype

for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

### Configure PEFT

In [22]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"]
)
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 13,631,488 || all params: 7,261,655,040 || trainable%: 0.1877


In [ ]:
len(model.peft_config)

1

### Load Dataset

In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer


dataset = load_dataset("json", data_files="data.json")["train"]

# Load Mistral tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Important if using padding

def format_mistral_prompt_with_context(example):
    context = example.get("context", "").strip()
    messages = example["messages"]
    
    prompt = ""
    if context:
        prompt += f"<s>[INST] Context: {context}\n"

    # Add user-assistant chat
    for i, msg in enumerate(messages):
        if msg["role"] == "user":
            if context and i == 0:
                prompt += f"User: {msg['content']} [/INST] "
            else:
                prompt += f"<s>[INST] {msg['content']} [/INST] "
        elif msg["role"] == "assistant":
            prompt += f"{msg['content']} </s>"

    return {"text": prompt.strip()}


# Format conversation into Mistral prompt
dataset = dataset.map(format_mistral_prompt)

# Tokenize
def tokenize(example):
    model_inputs = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,
        return_tensors="pt"
    )
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    return model_inputs

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)


ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.

### Tune the model

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=500,
    output_dir="./mistral-chat-lora"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

trainer.train()


### Evaluate

### Save the model